In [ ]:
base_experiment_path = 'data\\hp_configurations'

# Hyperparameter Optimization

## Component to optimize

In [ ]:
to_optimize_configuration_dict = None
to_optimize_configuration_path = None

### Use dict definition for component to optimize

In [ ]:

from automl.rl.whole_configurations import rl_multi_agent_pipeline as base_rl_configuration 

to_optimize_configuration_dict = base_rl_configuration.config_dict()
to_optimize_configuration_dict_input = to_optimize_configuration_dict["input"]

### Make sure component is properly seeded

In [ ]:
from automl.utils.collection_utils import substitute_value_in_dict


substitute_value_in_dict(to_optimize_configuration_dict_input, "do_full_setup_of_seed", True)

## Generate HP Optimization component

In [ ]:
hp_optimization_pipeline_path = None

### Generate Hp Suggestions

In [ ]:
from automl.hp_opt.hp_suggestion.hyperparameter_suggestions_generator import generate_hyperparameter_suggestion_list_for_config_dict
from automl.hp_opt.hp_suggestion.hyperparameter_suggestion import HyperparameterSuggestion


generated_hp_suggestions_list : list[HyperparameterSuggestion] = generate_hyperparameter_suggestion_list_for_config_dict(to_optimize_configuration_dict)

In [ ]:
import json

from automl.utils.json_utils.json_component_utils import ComponentValuesElementsEncoder


encoded_generated_hp_suggestions_list = json.dumps(generated_hp_suggestions_list, cls=ComponentValuesElementsEncoder, source_component=None, indent=4)

In [ ]:
print(encoded_generated_hp_suggestions_list)

### Hp Optimization pipeline input

In [ ]:
hp_optimization_input = {
    "n_trials" : 200,
    "steps" : 2,
    "base_directory" :  base_experiment_path,
    "hyperparameters_range_list" : generated_hp_suggestions_list
    }

### Evaluator

In [ ]:
evaluator = None

In [ ]:
from automl.rl.evaluators.rl_std_avg_evaluator import LastValuesAvgStdEvaluator
from automl.rl.evaluators.rl_evaluator_player import EvaluatorWithPlayer
from automl.rl.rl_player.rl_parallel_player import RLParallelPlayer

evaluator_class = EvaluatorWithPlayer

evaluator_input = {
    "rl_player_definition" : (RLParallelPlayer, {}),
    "base_evaluator" : (LastValuesAvgStdEvaluator, {"value_to_use" : "episode_steps"})
}

evaluator = (evaluator_class, evaluator_input)

In [ ]:
if evaluator != None:
    hp_optimization_input["evaluator_component"] = evaluator

### Pruner

In [ ]:
pruner = None

In [ ]:
pruner = 'PercentilePruner'
pruner_input = {"percentile" : 0.25}

In [ ]:
if pruner != None:
    hp_optimization_input["pruner"] = pruner

if pruner_input != None:
    hp_optimization_input["pruner_input"] = pruner_input

### To optimize config

In [ ]:
if to_optimize_configuration_dict != None:
    hp_optimization_input["configuration_dict"] = to_optimize_configuration_dict
    
    
if to_optimize_configuration_path != None: 
    hp_optimization_input["base_component_configuration_path"] = to_optimize_configuration_path

### If to seed

In [ ]:
hp_optimization_input["do_full_setup_of_seed"] = True

### Instantiate hp configuration

In [ ]:
from automl.hp_opt.hp_optimization_pipeline import HyperparameterOptimizationPipeline

if hp_optimization_pipeline_path != None: # load the configuration in the path
    
    from automl.utils.json_utils.json_component_utils import gen_component_from_path
    hp_optimization_pipeline = gen_component_from_path(hp_optimization_pipeline_path)

else: # if not, generate empty
    hp_optimization_pipeline = HyperparameterOptimizationPipeline()


hp_optimization_pipeline.pass_input(hp_optimization_input)


In [ ]:
hp_optimization_pipeline.setup_files() # partially initialize the hp optimization pipeline with the current input, it can be manually changed before it is used

In [ ]:
experiment_directory = hp_optimization_pipeline.get_artifact_directory()

In [ ]:
hp_optimization_pipeline.remove_input("base_component_configuration_path")

In [ ]:
hp_optimization_pipeline.save_configuration() # save the configuration as is to, even if the process is interrupted, we can see the added values

After this, we can make manual changes to the generated json files:

 - to_optimize_configuration.json, which is the base in which we want to optimize the hyperparameters of
 - configuration.json, the definition of the actual hyperparameter optimization experiment 
